In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout2d(0.1)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class FashionMNIST(nn.Module):
    def __init__(self):
        super(FashionMNIST, self).__init__()

        # Initial convolution
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        # Residual blocks
        self.layer1 = self.make_layer(64, 64, 2)
        self.layer2 = self.make_layer(64, 128, 2, stride=2)
        self.layer3 = self.make_layer(128, 256, 2, stride=2)

        # Global average pooling and classifier
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

    def make_layer(self, in_channels, out_channels, num_blocks, stride=1):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [3]:
def get_data_loaders(batch_size=64):
    transform = transforms.Compose([transforms.ToTensor()])
    train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

def save_logits(model, test_loader, device, studentID):
    model.eval()
    logits_list = []

    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            outputs = model(images)
            logits_list.append(outputs.cpu().numpy())

    logits = np.vstack(logits_list)
    np.save(f'{studentID}.npy', logits)


In [4]:
# Training function
def train_model(model, train_loader, test_loader, epochs=30, device="cuda"):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=0.01, epochs=epochs, steps_per_epoch=len(train_loader))

    best_acc = 0.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if batch_idx % 100 == 99:
                print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}, Loss: {running_loss/100:.3f}, '
                      f'Acc: {100.*correct/total:.2f}%')
                running_loss = 0.0

        model.eval()
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                test_total += targets.size(0)
                test_correct += predicted.eq(targets).sum().item()

        test_acc = 100. * test_correct / test_total
        print(f'Epoch: {epoch+1}, Test Accuracy: {test_acc:.2f}%')

        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), 'best_fashion_mnist.pth')

    return best_acc

In [ ]:
def main():
    studentID = "20233460"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader, test_loader = get_data_loaders()

    model = FashionMNIST().to(device)

    best_acc = train_model(model, train_loader, test_loader, device=device)

    save_logits(model, test_loader, device, studentID)

    print(f'Best Test Accuracy: {best_acc:.2f}%')

if __name__ == '__main__':
    main()

100%|██████████| 26421880/26421880 [00:12<00:00, 2148083.90it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 209601.58it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:06<00:00, 722952.65it/s] 


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 16101623.41it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch: 1, Batch: 100, Loss: 1.161, Acc: 58.48%
Epoch: 1, Batch: 200, Loss: 0.715, Acc: 66.12%
Epoch: 1, Batch: 300, Loss: 0.609, Acc: 70.24%
Epoch: 1, Batch: 400, Loss: 0.530, Acc: 73.14%
Epoch: 1, Batch: 500, Loss: 0.500, Acc: 75.00%
Epoch: 1, Batch: 600, Loss: 0.475, Acc: 76.29%
Epoch: 1, Batch: 700, Loss: 0.457, Acc: 77.34%
Epoch: 1, Batch: 800, Loss: 0.449, Acc: 78.19%
Epoch: 1, Batch: 900, Loss: 0.436, Acc: 78.91%
Epoch: 1, Test Accuracy: 85.47%
Epoch: 2, Batch: 100, Loss: 0.407, Acc: 85.44%
Epoch: 2, Batch: 200, Loss: 0.366, Acc: 86.34%
Epoch: 2, Batch: 300, Loss: 0.391, Acc: 86.26%
Epoch: 2, Batch: 400, Loss: 0.389, Acc: 86.22%
Epoch: 2, Batch: 500, Loss: 0.390, Acc: 86.25%
Epoch: 2, Batch: 600, Loss: 0.354, Acc: 86.44%
Epoch: 2, Batch: 700, Loss: 0.346, Acc: 86.67%
